In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 8.1 MB/s 
     |████████████████████████████████| 636 kB 59.2 MB/s 
     |████████████████████████████████| 895 kB 59.0 MB/s 
     |████████████████████████████████| 3.3 MB 53.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import  DistilBertTokenizer, AdamW, get_linear_schedule_with_warmup,DistilBertForSequenceClassification

import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import bz2
import os

import numpy as np

In [ ]:
if torch.cuda.is_available():
  device=torch.device('cuda:0')
  print('Cuda')
else:
  device=torch.device('cpu')
  print('cpu')


Cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_file = bz2.BZ2File('drive/MyDrive/amazon/train.ft.txt.bz2')
test_file = bz2.BZ2File('drive/MyDrive/amazon/test.ft.txt.bz2')

In [ ]:
train_file_lines = train_file.readlines()
test_file_lines=test_file.readlines()

In [ ]:
train_file_lines = [x.decode('utf-8') for x in train_file_lines]
test_file_lines = [x.decode('utf-8') for x in test_file_lines]

In [ ]:
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file_lines[:50000]]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file_lines[:50000]]

In [ ]:
test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_file_lines[:4000]]
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_file_lines[:4000]]

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')



In [ ]:
train_encodings = tokenizer(train_sentences,
                            add_special_tokens=True,
                            truncation=True,
                            return_token_type_ids=False,
                            padding=True,
                            return_attention_mask=True,
                            return_tensors='pt')
test_encodings=tokenizer(test_sentences,add_special_tokens=True,
                         truncation=True,
                        return_token_type_ids=False,
                         padding=True,
                            return_attention_mask=True,return_tensors='pt')

In [ ]:
class AmazonDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return {
            # 'review': self.sentence[idx],
            'input_ids' : item['input_ids'].flatten(),
            'attention_mask' : item['attention_mask'].flatten(),
            'labels': torch.tensor(item['labels'],dtype=torch.long)
        }

    def __len__(self):
        return len(self.labels)


train_dataset = AmazonDataset(train_encodings, train_labels)
test_dataset = AmazonDataset(test_encodings, test_labels)

In [ ]:
train_dataset.__getitem__(2)

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0

In [ ]:
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=15,shuffle=False)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased',num_labels=2)
model=model.to(device)
model

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.bias', 'classifie

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [ ]:
import time
optimizer=AdamW(model.parameters(),lr=5e-5)
criterion=nn.CrossEntropyLoss().to(device)
epochs=3
scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,
                                                num_training_steps=len(train_loader) * epochs)

In [ ]:
from tqdm.auto import tqdm
def train(model,train_loader,loss_f,optim,scheduler,num_of_epochs,path,val_dataloader=None,evalution=False):
  try:
    os.mkdir(path)
    print("Created folder for models")
  except:
    path=path

  print('Training Started \n')
  for epoch in range(num_of_epochs):
    trn_corr,tst_corr=0,0
    train_losses=[]
    min_acc=80
    model.train()
    
    for step,batch in enumerate(train_loader):
      step+=1
      input_ids=batch['input_ids'].to(device)
      attention_mask=batch['attention_mask'].to(device)
      labels=batch['labels'].to(device)
      model.zero_grad()
      logits=model(input_ids, attention_mask)
      loss=loss_f(logits['logits'],labels)
      predicted=torch.max(logits['logits'],1)[1]
      trn_corr+= (predicted==labels).sum()
      train_losses.append(loss.item())
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      optim.step()
      scheduler.step()

    tr_loss=np.mean(train_losses)
    tr_acc=trn_corr.item()*100/(32*step)


    if evalution == True:
      val_loss, val_acc=evaluate(model,val_dataloader,loss_f)
      if val_acc > min_acc:
        min_acc= val_acc
        torch.save(model.state_dict(),f"{path}/val_acc {val_acc}.pt")
      

      print(f"epoch : {epoch+1}, training Loss : {tr_loss}, training ACC : {tr_acc}, val loss : {val_loss}, val ACC : {val_acc}" )
    print('\n')
  print('Training Complete')




  


  

In [ ]:
def evaluate(model,val_loader,loss_f):
  print('evaluation Started ')
  model.eval()
  val_accuracy=[]
  val_loss=[]
  
  for step,batch in enumerate(val_loader):
    val_input_ids= batch['input_ids'].to(device)
    val_attention_mask=batch['attention_mask'].to(device)
    val_labels=batch['labels'].to(device)

    with torch.no_grad():
      logits=model(val_input_ids, val_attention_mask)
    loss=loss_f(logits['logits'],val_labels)
    val_loss.append(loss.item())
    preds=torch.argmax(logits['logits'],dim=1).flatten()
    val_acc_=(preds==val_labels).cpu().numpy().mean()*100
    val_accuracy.append(val_acc_.item())
  val_loss=np.mean(val_loss)
  val_acc=np.mean(val_accuracy)
  
  return val_loss,val_acc

In [ ]:
train(model,train_loader,criterion,optimizer,scheduler,epochs,'drive/MyDrive/models',test_loader,True)

Created folder for models
Training Started 

evaluation Started 
epoch : 1, training Loss : 0.12806707928261063, training ACC : 95.57941458733205, val loss : 0.1870469259270833, val ACC : 93.68289637952559


evaluation Started 
epoch : 2, training Loss : 0.06635476997561911, training ACC : 98.12859884836853, val loss : 0.25380308627000964, val ACC : 93.5330836454432


evaluation Started 
epoch : 3, training Loss : 0.0428696322728244, training ACC : 98.8863563659629, val loss : 0.25380308627000964, val ACC : 93.5330836454432


Training Complete


In [ ]:
import glob
from transformers import  DistilBertTokenizer,AutoModelForMaskedLM,AutoConfig
import torch
# from google.colab import drive
# drive.mount('/content/drive')

tokenizer = DistilBertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased')

model.load_state_dict(torch.load('/content/drive/MyDrive/models/val_acc 93.68289637952559.pt'))





The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.
Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (i

<All keys matched successfully>

In [ ]:
model.eval()
sentence = input()

tokenized=tokenizer(sentence)
logits=model(torch.tensor(tokenized['input_ids']).unsqueeze(0), torch.tensor(tokenized['attention_mask']).unsqueeze(0))
print(tokenized)
print(torch.argmax(logits['logits'],dim=1).flatten())

Had this now about 3 months and it just cracked. Wish it was more durable.
{'input_ids': [101, 10407, 10372, 11628, 10935, 124, 15781, 10110, 10197, 12125, 86199, 10390, 119, 33020, 10197, 10140, 10772, 83410, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
tensor([0])


In [ ]:
sentence = input()

tokenized=tokenizer(sentence)
logits=model(torch.tensor(tokenized['input_ids']).unsqueeze(0), torch.tensor(tokenized['attention_mask']).unsqueeze(0))
print(tokenized)
print(torch.argmax(logits['logits'],dim=1).flatten())

seller was awesome! Thank you!
{'input_ids': [101, 83815, 10140, 37079, 42279, 10688, 106, 67715, 10855, 106, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
tensor([1])
